In [12]:
import requests
import datetime
import matplotlib.pyplot as plt
# import plotly.graph_objects as go
import plotly.offline as py
import plotly
from plotly.offline import init_notebook_mode
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

py.init_notebook_mode(connected=True)
import numpy as np

#serie de mares de ubatuba em 1981
link = "https://www.ime.usp.br/~pam/h-ubatuba81"
f = requests.get(link)

Fs = 150.0;  # sampling rate
Ts = 1.0/Fs; # sampling interval

a = f.text.split("\n")
a = a[:-1]
a = list(map(int, a))

n = len(a) # length of the signal

x = np.arange(n)
y = np.asarray(a)

T = n/Fs

frq = x/T

data=[go.Scatter(x=x, y=y, mode='lines')]
py.iplot(data)

Y = np.fft.fft(y)/n

data=[go.Scatter(x=frq, y=abs(Y), mode='lines')]
py.iplot(data)

## O primeiro gráfico é a série temporal

## Já o segundo gráfico é o espectro da FFT
### Nele podemos perceber picos nas frequências : 12.1875 e 137.8125.